# Problem 1

## `infix_to_prefix(expression)`:

We're used to seeing most mathematical expressions in _infix_ notation, which places the operator between its two operands (e.g. `(A + B)` or in general `(A op B)`). While this may be intuitive for us, it is more difficult for a computer to parse this notation. This is why some programming languages use _prefix_ notation, which places the operator before its operands (e.g. `(+ A B)` or in general `(op A B)`).

Given a `string` representing an expression in _infix_ notation, return a `string` representing the same expression converted to _prefix_ notation.

You may assume that all variables are a single letter and all numbers are non-negative `int`s. You may also assume that the expression is fully parenthesized.

Examples:

* `infix_to_prefix("x")` should return `"x"`
* `infix_to_prefix("(x + 3)")` should return `"(+ x 3)`
* `infix_to_prefix("((3 * x) + (4 * y))")` should return `"(+ (* 3 x) (* 4 y))"`

In [1]:
def infix_to_prefix(expression):        
    def tokenize(str_list):
        '''
        Separates the parantheses from the given str_list to separate parens 
        from the meaningful tokens
        '''
        new_str_list = []
        for e in str_list:
            #Recursive cases: parens are combined with tokens. separate them
            if e[0] == '(':
                new_str_list.extend([e[0]] + tokenize([e[1:]]))
            elif e[-1] == ')':
                new_str_list.extend(tokenize([e[:-1]]) + [e[-1]])
            #This is a meaningful token. add it to our result list
            else:
                new_str_list.append(e)
        return new_str_list
    
    def parse(tokens):
        def parse_expression(index):
            token = tokens[index]
            #Base Case
            #Token is an integer or a variable.
            if token.isdigit() or token.isalpha():
                return token, index + 1\
            #The expression is in the form of (E1 op E2) --> Recursively parse 
            #E1 and E2 then return the corresponding infix expression with the index of next
            #rightmost paren
            else:
                #Token is '(', create e1
                if token == '(':
                    e1, index = parse_expression(index + 1)
                    token = tokens[index]

                #Token is an operator, create e2
                if token in '+-*/':
                    op = token
                    e2, index = parse_expression(index + 1)
                
                prefix_not = '(' + op + ' ' + e1 + ' ' + e2 + ')'
                return prefix_not, index + 1
        parsed_expression, next_index = parse_expression(0)
        return parsed_expression
    tokens = tokenize(expression.split(sep = ' '))
    return parse(tokens)
    
    

In [13]:
test1 = infix_to_prefix("x") == "x"
test2 = infix_to_prefix("(x + 3)") == "(+ x 3)"
test3 = infix_to_prefix("((3 * x) + (4 * y))") == "(+ (* 3 x) (* 4 y))"
if all([test1, test2, test3]):
    print('All tests are passed')
    print(infix_to_prefix("x"))
    print(infix_to_prefix("(x + 3)"))
    print(infix_to_prefix("((3 * x) + (4 * y))"))
else:
    print('Some tests failed')

All tests are passed
x
(+ x 3)
(+ (* 3 x) (* 4 y))


# Problem 2

## `n_bishops(n, bishop_locs, target)`:

Given an `n`x`n` board, a valid placement of bishops on a chess board is one such that no two bishops can attack each other. In chess, bishops can move any distance diagonally. This means that no two bishops can be directly diagonal from each other.

Within the board, there may be bishops already present that **cannot be moved.** The locations of these bishops are stored in `bishop_locs`. The board is oriented such that `(0, 0)` is the cell in the top-left corner and `(0, n-1)` is the cell in the top-right corner.

Given a `target` number of bishops that we want on the board, find a way to place the remaining bishops on the board such that the resulting board is valid and contains `target` number of bishops. Return a set containing the locations of all bishops on the resulting board. In the case that no valid placement of bishops is possible, return `None`.

You may assume that we will only call this with existing set-ups of bishops where no preexisting bishops can attack each other.

**Hint:** Recall from `n_queens` in lecture that given a bishop in `(row1, col1)` and another in `(row2, col2)`, checking whether one can attack the other can be done by simply checking if `abs(row1 - row2) == abs(col1 - col2)`.

Examples:

* If `n = 3`, `bishop_locs = {}`, and `target = 4`, one of many correct assignments is `{(0, 0), (0, 1), (0, 2), (2, 1)}`

![valid_bishop_1](files/valid_bishop_1.png)

* If `n = 3`, `bishop_locs = {(1, 0), (1, 2)}`, and `target = 4`, one of many correct assignments is `{(1, 0), (1, 2), (2, 0), (2, 2)}`. (The black bishops are bishops that cannot be moved)

![valid_bishop_2](files/valid_bishop_2.png)

* If `n = 2`, `bishop_locs = {}`, and `target = 3`, `None` should be returned, because it is impossible to place 3 bishops on a 2x2 board while keeping all bishops safe from attacks from all other bishops.

![invalid_bishop_1](files/invalid_bishop_1.png)

In [33]:
def n_bishops(n, bishop_locs, target):
    # Helper based on the hint above
    def can_attack(bishop1, bishop2):
        row1 = bishop1[0]
        col1 = bishop1[1]
        row2 = bishop2[0]
        col2 = bishop2[1]
        return abs(row1 - row2) == abs(col1 - col2)
    
    def helper(grid, unvisited, current_target):
        #Base case: All of the bishops are placed
        if current_target == 0:
            return bishop_locs
        
        elif len(unvisited) == 0 and current_target > 0:
            return None
        #Recursive step: 
        else:
            for i in range(len(unvisited)):
                bishop1 = unvisited[i]
                row = bishop1[0]
                col = bishop1[1]
                any_attacks = all([not can_attack(bishop1, bishop2) for bishop2 in bishop_locs])
                if any_attacks:
                    bishop_locs.add(bishop1)
                    grid[row][col] = 1
                    if helper(grid, unvisited[:i] + unvisited[i+1:], current_target - 1) is not None:
                        return bishop_locs
                    bishop_locs.remove(bishop1)
                    grid[row][col] = -1
            return None
        
    def create_grid(locs):
        '''
        By using giving locations, creates a nxn grid
        '''
        grid = [[-1 for i in range(n)] for j in range(n)]
        for loc in locs:
            row = loc[0]
            col = loc[1]
            grid[row][col] = 1
        
        unvisited = []
        
        for row in range(n):
            for col in range(n):
                if grid[row][col] == -1:
                    unvisited.append((row, col))
        return grid, unvisited
    
    grid, unvisited = create_grid(bishop_locs)
    return helper(grid, unvisited, target - len(bishop_locs))
            
    

In [34]:
print(n_bishops(3, set(), 4))
print(n_bishops(3, {(1, 0), (1, 2)}, 4))
print(n_bishops(2, set(), 3))

{(0, 1), (0, 0), (0, 2), (2, 1)}
{(1, 2), (1, 0), (0, 0), (0, 2)}
None


# Problem 3

## `pipe_cutting(requests, stock_length)`:

You are tasked with buying plumbing pipes for a construction project. Your foreman gives you a list of the varying lengths of pipe needed. The local hardware store sells pipes in one fixed length but has a saw for you to cut up the pipes in any way you choose. Your job is to figure out the minimum number of stock pipes required to satisfy the list of requests, in order to save money and minimize waste.

Please implement `pipe_cutting(requests, stock_length)` where `requests` is a list of pipe lengths needed for the project and `stock_length` is the length of the pipes you can purchase at Home Depot. `pipe_cutting` should return an `int` giving the minimum number of purchased pipes needed to satisfy the list of requests.

You can assume that all elements in `requests` are positive numbers that are no longer than the stock pipe length. So, in the worst case, it will take a number of stock pipes equal to the size of the request list. You do not need to report the cutting/division in the optimal solution, just report the minimum number of stock pipes needed.

There are different ways of solving this problem. We recommend, but do not require, a recursive approach for which the hint below will be useful.

**Hint:** For any particular request, the request might be satisfied by choosing a left-over pipe length or cutting from any left-over pipe lengths, or by choosing to buy a new pipe and cutting it to length.

Examples:

* `pipe_cutting([], 8)` returns `0`
* `pipe_cutting([7], 7)` returns `1`
* `pipe_cutting([7, 6, 4], 10)` returns `2` (the cuts are `[7], [6, 4]`)
* `pipe_cutting([4, 3, 4, 1, 7, 8], 10)` returns `3` (the cuts are `[4, 4, 1], [3, 7], [8]`)

In [1]:
def pipe_cutting(requests, stock_length):
    def helper(order_list, number_of_pipes, left_over_pipes):
        #Base case: Trading is done, no requests left
        if len(order_list) == 0:
            return number_of_pipes
        
        #Trading continues
        else:
            request = order_list[0]
            #Buy a new pipe
            if stock_length > request:
                selection1 = helper(order_list[1:], number_of_pipes + 1, left_over_pipes + [stock_length - request])
            else:
                selection1 = helper(order_list[1:], number_of_pipes + 1, left_over_pipes)
                
            #Or choose from left overs
            selection2 = len(requests)
            selections = []
            for i in range(len(left_over_pipes)):
                if request <= left_over_pipes[i]:
                    selections.append(helper(order_list[1:], number_of_pipes, left_over_pipes[:i] + [left_over_pipes[i] - request] + left_over_pipes[i+1:]))
            
            if len(selections) > 0:
                selection2 = min(selections)
            
            return min(selection1, selection2)
                          
    return helper(requests, 0, [])

In [2]:
print(pipe_cutting([], 8))
print(pipe_cutting([7], 7))
print(pipe_cutting([7, 6, 4], 10))
print(pipe_cutting([4, 3, 4, 1, 7, 8], 10))

0
1
2
3
